Transcript Files

The coding below is made to list a specific emotion in what's said in these transcripts throughout Season 21 and 22

In [3]:
import spacy
import os, shutil
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString
import pandas as pd
import json

# nlp = spacy.cli.download("en_core_web_md")
# nlp = spacy.load('en_core_web_md')

workingDir = os.getcwd()
print("current working directory: " + workingDir)

insideDir = os.listdir(workingDir)
print("inside this directory are the following files AND directories: " + str(insideDir))

CollPath = os.path.join(workingDir, 'TXT_files')
print(CollPath)


def readTextFiles(filepath):
    with open(filepath, 'r', encoding='utf8') as f:
        readFile = f.read()
       
        stringFile = str(readFile)
        lengthFile = len(readFile)
        print(lengthFile)
       
        tokens = nlp(stringFile)
        
        vectors = tokens.vector

        wordOfInterest = nlp(u'joking')
       
        highSimilarityDict = {}
        for token in tokens:
            if (token and token.vector_norm):
                if wordOfInterest.similarity(token) > .3:
                    highSimilarityDict[token] = wordOfInterest.similarity(token)
        print("This is a dictionary of words most similar to the word " + wordOfInterest.text + " in this file.")
        print(highSimilarityDict)
        switcheroo = {val: key for key, val in highSimilarityDict.items()}
        deduped = {val: key for key, val in switcheroo.items()}
        print(str(len(switcheroo)) + ' **** ' + f'{switcheroo=}')

        print(len(deduped), ' **** ', f'{deduped=}')
        print(len(deduped.items()), " vs ", len(highSimilarityDict.items()))

        highSimilarityReduced = {}
        for key, value in highSimilarityDict.items():
            if value not in highSimilarityReduced.values():
                highSimilarityReduced[key] = value
        print(highSimilarityReduced)
        print(len(highSimilarityReduced.items()), " vs ", len(highSimilarityDict.items()))


        sortedSimValues = sorted(deduped.items(), key=lambda x: x[1], reverse=True)
        print(type(sortedSimValues), f'{sortedSimValues=}')

        sortedSimDict = dict(sortedSimValues)
        print(type(sortedSimValues), f'{sortedSimValues=}')

        return sortedSimDict


# ebb: This controls our file handling as a for loop over the directory:
with open('similarityReadings.txt', 'a', encoding='utf8') as f:
    for file in sorted(os.listdir(CollPath)):
        if file.endswith(".txt"):
            filepath = f"{CollPath}/{file}"
            print(filepath)
            similarityData = readTextFiles(filepath)
            f.write(filepath + '\n')
            f.write(str(similarityData) + '\n\n')
    if os.path.exists("JSON-output"):
        shutil.rmtree("JSON-output")
    if os.path.exists("csv-output"):
        shutil.rmtree("csv-output")
    if os.path.exists("xml-output"):
        shutil.rmtree("xml-output")
    os.mkdir('JSON-output')
    os.mkdir('csv-output')
    os.mkdir('xml-output')

    df = pd.DataFrame.from_dict(similarityData.items(), orient="columns")
    df.columns = ['token', 'similarity']
    print(df)
    df.to_csv(f'csv-output/{file}.tsv', sep='\t', index=False, encoding='utf-8')

if os.path.exists("JSON-output"):
    shutil.rmtree("JSON-output")
if os.path.exists("csv-output"):
    shutil.rmtree("csv-output")
if os.path.exists("xml-output"):
    shutil.rmtree("xml-output")

os.mkdir('JSON-output')
os.mkdir('csv-output')
os.mkdir('xml-output')

for file in sorted(os.listdir(CollPath)):
    if file.endswith(".txt"):
        filepath = f"{CollPath}/{file}"
        print(filepath)
        filenameTxt = os.path.basename(filepath).split('/')[-1]
        filename = filenameTxt[:-4]
        print(filename)
        similarityData = readTextFiles(filepath)

        stringKeys = {str(key): val for key, val in similarityData.items()}
        print(f'{stringKeys=}')
        with open(f'JSON-output/{filename}.json', 'w') as fp:
            JSON = json.dumps(stringKeys)
            print(f'{JSON=}')
            json.dump(stringKeys, fp)
        stringKeys = {str(key): val for key, val in similarityData.items()}
        print(f'{stringKeys=}')
        with open(f'JSON-output/{filename}.json', 'w') as fp:
            JSON = json.dumps(stringKeys)
            print(f'{JSON=}')
            json.dump(stringKeys, fp)
        df = pd.DataFrame.from_dict(similarityData.items(), orient="columns")
        df.columns = ['token', 'similarity']
        print(df)
        df.to_csv(f'csv-output/{filename}.tsv', sep='\t', index=False, encoding='utf-8')

        xml = dicttoxml(similarityData)
        dom = parseString(xml)
        print(dom.toprettyxml())
        with open(f'xml-output/{filename}.xml', 'w') as xmlFile:
            xml_decode = xml.decode()
            xmlFile.write(xml_decode)
            xmlFile.close()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 36.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
current working directory: /Users/liz/Documents/FamilyGuy/Jupyter
inside this directory are the following files AND directories: ['similarityReadings.txt', '.ipynb_checkpoints']
/Users/liz/Documents/FamilyGuy/Jupyter/TXT_files


FileNotFoundError: [Errno 2] No such file or directory: '/Users/liz/Documents/FamilyGuy/Jupyter/TXT_files'